### Convert the MS2 data to feature layer for Sun Cloud area.
MS2 stations shapefile and counts Excel files obtained from Sage Donaldson and Lucas Murray at AZDOT. 

In [ ]:
# define workspace
import arcpy
import os
db = arcpy.env.workspace = './data/sun_cloud_ms2_revision.gdb'
arcpy.Describe(db)

data_dir = '.\data'
full_data_path = os.path.abspath(data_dir)
full_data_path

Custom functions. 

In [ ]:
from arcgis.gis import GIS
gis = GIS('pro')
from arcgis.features import FeatureLayer

# save feature layer to db
def save_fl(db, url, outname):
    fl = FeatureLayer(url)
    featureset = fl.query()
    featureset.save(db, outname)

# project in_data to match the target_data

def project(in_data, target_data, out_name):
    #TODO: do not hardcode this
    target_data = 'PM3_For_HPMS_2_ExportFeature'

    in_sr = arcpy.Describe(in_data).spatialReference
    target_sr = arcpy.Describe(target_data).spatialReference
   
    if len(arcpy.ListTransformations (in_sr, target_sr))>0:
        datum_conversion = arcpy.ListTransformations (in_sr, target_sr)[0]
    else: 
        datum_conversion = ''
    arcpy.Project_management(
        in_dataset = in_data, 
        out_dataset = out_name,
        out_coor_system = target_sr,
        transform_method = datum_conversion)

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})


def get_average(lst):
    return sum(lst) / len(lst)

Import the source spreadsheet to file geodatabase (this cell will run for a few minutes!)

In [ ]:
# import source spreadsheet to file geodatabase
import re
import pandas as pd

# import excel files to file geodatabase
data_dir = '.\data'
full_data_path = os.path.abspath(data_dir)

for file in os.listdir(data_dir):
    if(file.lower().endswith('.xlsx')):
        excel_file = os.path.join(full_data_path, file)
        out_name = re.sub(r'[^a-zA-Z0-9]', '', os.path.splitext(file)[0])
        new_name = os.path.join(full_data_path, out_name +'.xlsx')
        print(new_name)
        # rename the excel files
        os.rename(excel_file, new_name)

        # use full file path
        new_file = os.path.join(full_data_path, new_name)
        out_tb = os.path.join(os.path.abspath(db), out_name)
        print(new_file, out_tb)

        arcpy.conversion.ExcelToTable(new_file, out_tb)

                    

Merge the tables into a single table.

In [ ]:
tbs = arcpy.ListTables()
merged_name = 'ms2_merge_all'
arcpy.management.Merge(tbs, merged_name)


In [ ]:
# save county layer locally
fc='sc_boundary'
sc_county_url = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/ArcGIS/rest/services/Sun_Cloud_Jurisdictional_Boundaries/FeatureServer/5'
if arcpy.Exists(fc)==False:
    save_fl(db, sc_county_url, fc)


Clip the station layer to Sun Cloud boundary. 

In [ ]:
arcpy.analysis.Clip('MS2_Stations', 'sc_boundary', 'ms_stations_clip')

In [ ]:
# save the routes layer locally
routes_url = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Sun_Cloud_Routes/FeatureServer/92'
save_fl(db, routes_url, 'sc_routes')

Snap the station to the sun cloud routes. 

In [ ]:

arcpy.edit.Snap("ms_stations_clip", "sc_routes EDGE '60 Meters'")

# TODO: Visually check the snapping result in the map.

Delete the stations that did not snap.

In [ ]:
selected = arcpy.management.SelectLayerByLocation(
    in_layer="ms_stations_clip",
    overlap_type="INTERSECT",
    select_features="sc_routes",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="INVERT"
)
count = arcpy.GetCount_management(selected)[0]
print(count)

if int(count)>0:
    arcpy.DeleteFeatures_management(selected)

In [ ]:
! pip install fuzzywuzzy
# ! pip install python-Levenshtein

Add a field to note snap validation result.

In [ ]:
fc = 'ms_stations_clip' 
arcpy.AddField_management(fc, 'flag_num', 'LONG')

Compare the On(road) and RouteID value to the segment route_id.  Get max value of partial match value. (~9,000 rows)

In [ ]:
from fuzzywuzzy import fuzz

route_layer = 'sc_routes'
# fields in source and destination layers
fields = ['SHAPE@', 'RouteID', 'flag_num', 'OID@', 'On']
segment_name_field = 'route_id'
# create a memory layer
arcpy.Delete_management('point_lyr')
arcpy.management.MakeFeatureLayer('ms_stations_clip', 'point_lyr')


with arcpy.da.UpdateCursor('point_lyr', fields) as cursor:
    for row in cursor:
        print('oid', row[3])
        name_in_pt = row[1]
        on_in_pt = row[4]
        print(name_in_pt)
        arcpy.Delete_management('line_lyr')
        arcpy.management.MakeFeatureLayer(route_layer, 'line_lyr')
        selected = arcpy.management.SelectLayerByLocation(
            in_layer='line_lyr',
            overlap_type='INTERSECT', 
            select_features=row[0], 
            selection_type='NEW_SELECTION')
        
        if (int(arcpy.GetCount_management(selected)[0])>0):
            road_name = list(r[0] for r in arcpy.da.SearchCursor(selected, segment_name_field))[0]
            candidates = []
            score = fuzz.partial_ratio(name_in_pt.upper().strip(), road_name.upper().strip())
            candidates.append(score)
            print(score)
            score2 = fuzz.partial_ratio(on_in_pt.upper().strip(), road_name.upper().strip())
            candidates.append(score2)
            print(candidates)
            row[2]= max(candidates)
            cursor.updateRow(row) 

Review in the map flag_num < 90.  determine threshold for deletion. 

Review the fuzzy string match result in the map.  Determine determine the score for deletion.

In [ ]:
# delete invalid snapped points
selected = arcpy.management.SelectLayerByAttribute('ms_stations_clip', 'NEW_SELECTION', 'flag_num < 80 or flag_num is null')
count = int(arcpy.GetCount_management(selected)[0])
print(count)
if(count>0):
    arcpy.DeleteFeatures_management(selected)


Delete identical. (34,5050 -> 9,404)

In [ ]:
arcpy.management.DeleteIdentical(
    in_dataset="ms_stations_clip",
    fields="LocalId",
    xy_tolerance=None,
    z_tolerance=0
)

### TODO: manually review scores 80-89. move points if necessary. 

Pare down the volume count data. 
- If no match is found in the station.
- Weekend counts.

Get LocalIDs in the station layer. 

In [ ]:
ids = unique_values('ms_stations_clip', 'LocalId')
# _list = ','.join(str(e) for e in ids)
_list = tuple(ids)
_list


In [ ]:
# from the merged table, delete rows if ID is not in the unique ID list (i.e. no matching station)
sql = 'Location_ID not in {0}'.format(_list)
# print(sql)
selected = arcpy.SelectLayerByAttribute_management('ms2_merge_all', 'NEW_SELECTION', sql)
count = int(arcpy.GetCount_management(selected)[0])
print(count)
if(count>0):
    arcpy.DeleteRows_management(selected)

Add fields to the station layer.

In [ ]:
# add fields 2010 through 2022
year_fields = []
i = 2010
while i < 2023:
    name = 'y_{}'.format(i)
    year_fields.append(name)
    arcpy.management.AddField('ms_stations_clip', name, 'DOUBLE')
    i+=1
year_fields

In [ ]:
# add field for weekened eval
tb = 'ms2_merge_all'
arcpy.management.AddField(tb, 'is_weekend', 'TEXT')

Custom functions for identifying weekend dates.

In [ ]:
from datetime import date


def is_weekend(_date):
    week_num = _date.weekday()
    if week_num < 5:
        return False
    else:  # 5 Sat, 6 Sun
        return True


def weekend_eval(table, start_date):
    """
    Assess if either date falls on a weekend day. If yes, update the attribute
    :param table: input table
    :param start_date: start date field name
    :param end_date: end date field name
    :return: None
    """
    fields = [start_date, 'is_weekend']
    with arcpy.da.UpdateCursor(table, fields) as cursor:
        for row in cursor:
            start = is_weekend(row[0])
            # end = is_weekend(row[1])
            if start:
                row[1] = 'yes'
            cursor.updateRow(row)
    # del row, cursor


In [ ]:
date_field = 'Volume_Count_Start_Date'
weekend_eval('ms2_merge_all', date_field)

Delete the weekend counts from the merged table.

In [ ]:
# delete weekend records
tb = 'ms2_merge_all'
selected = arcpy.SelectLayerByAttribute_management(tb, 'NEW_SELECTION', "is_weekend = 'yes'")
cnt = int(arcpy.GetCount_management(selected).getOutput(0))
print(cnt)
if cnt > 0:
    arcpy.AddMessage('Deleting {} rows ...'.format(cnt))
    arcpy.DeleteRows_management(selected)
    

Calculate average count for each year from the merged table.

In [ ]:
# delete ramp stations. 
selected = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="ms_stations_clip",
    selection_type="NEW_SELECTION",
    where_clause="Dir = 'RAMP'",
    invert_where_clause=None
)

arcpy.DeleteFeatures_management(selected)

In [ ]:
# get unique id list
ids = unique_values('ms_stations_clip', 'LocalId')
print(ids)
total_length = len(list(set(ids)))
print(total_length)

(~ 150 minutes)

In [ ]:
arcpy.Delete_management('fc_layer')
fc = 'ms_stations_clip'
arcpy.MakeFeatureLayer_management(fc, 'fc_layer')

arcpy.Delete_management('tb_view')
tb = 'ms2_merge_all'
arcpy.MakeTableView_management(tb, 'tb_view')
i = 1
for id in ids:
    print('{0}/{1}'.format(i, total_length))
    sql = "Location_ID = '{}'".format(id)
    print(sql)
    dates = [row[0] for row in arcpy.da.SearchCursor(
        in_table='tb_view', 
        field_names='Volume_Count_Start_Date',
        where_clause=sql
        )]
    years = [item.year for item in dates]
    unique_years = (list(set(years)))
    
    # for each year valus with counts, get average. 
    unique_years = sorted(unique_years)
    print(unique_years)
    for y in unique_years:
        date_sql = sql + "and Volume_Count_Start_Date BETWEEN date '{0}-01-01' AND date '{0}-12-31'".format(y)
        # print(date_sql)
        values = [row[0] for row in arcpy.da.SearchCursor(
        in_table='tb_view', 
        field_names='Volume_Count_Total',
        where_clause=date_sql
        )]
        if(len(values)>0):
            avg = get_average(values)
            year_field = 'y_{}'.format(y)
            _query="LocalId = '{}'".format(id)

            with arcpy.da.UpdateCursor('fc_layer', [year_field], _query ) as cursor:
                for row in cursor:
                    print(avg)
                    row[0] = round(avg)
                    cursor.updateRow(row)
        i=i+1


Join the count and station and export. 

In [ ]:
# join the station to table and export
joined = arcpy.management.AddJoin("ms_stations_clip", "LocalId", "ms2_merge_all", "Location_ID", "KEEP_COMMON", "INDEX_JOIN_FIELDS")
arcpy.conversion.ExportFeatures(joined, "sun_cloud_ms2")

Standardize the field names.

In [ ]:
# # standadize field names
# fields = arcpy.ListFields('sun_cloud_ms2')
# req_fields = [field.name for field in fields if field.required is True]
# req_fields

In [ ]:
fields = arcpy.ListFields('ms_stations_clip')
req_fields = [field.name for field in fields if field.required is True]
req_fields

In [ ]:
keep_fields = ['LocalId', 'On', 'From_','To', 'Dir','LatestCnt','LatestDate','Agency','RouteID',  
 'y_2010',
 'y_2011',
 'y_2012',
 'y_2013',
 'y_2014',
 'y_2015',
 'y_2016',
 'y_2017',
 'y_2018',
 'y_2019',
 'y_2020',
 'y_2021',
 'y_2022',
 ]

In [ ]:
final_fields = keep_fields + req_fields
for f in fields:
    if f.name not in final_fields:
        print(f.name)
        arcpy.management.DeleteField ('ms_stations_clip', f.name)

In [ ]:
field_dict = [  {'name':'LocalId', 'new_name':'local_id', 'alias':'Local ID'}, 
                {'name':'On','new_name':'on_road','alias':'On'},
                {'name':'From_', 'new_name':'from_road', 'alias':'From Road'}, 
                {'name':'To', 'new_name':'to_road', 'alias':'To Road'}, 
                {'name':'_dir', 'new_name':'dir', 'alias':'Dir'}, 
                {'name':'LatestCnt', 'new_name':'latest_estimate', 'alias':'Latest Estimate'}, 
                {'name':'LatestDate', 'new_name':'latest_estimate_date', 'alias':'Latest Estimate Date'}, 
                {'name':'Agency', 'new_name':'agency', 'alias':'Agency'}, 
                {'name':'RouteID', 'new_name':'route_id', 'alias':'Route ID'}
                ]
for item in field_dict:
    # print(item.get('name'))
    old_name = item.get('name')
    new_name = item.get('new_name')
    alias = item.get('alias')
    if old_name.lower() == new_name:
        new_name = '_{}'.format(new_name)
    print(old_name, new_name)


    arcpy.management.AlterField(in_table='ms_stations_clip', 
                                field=old_name, 
                                new_field_name = new_name,
                                new_field_alias = alias)

In [ ]:
field_dict = [  {'name':'_dir', 'new_name':'dir', 'alias':'Dir'}, 
                {'name':'latest_count', 'new_name':'latest_estimate', 'alias':'Latest Estimate'}, 
                {'name':'latest_count_Date', 'new_name':'latest_estimate_date', 'alias':'Latest Estimate Date'}
                ]
for item in field_dict:
    # print(item.get('name'))
    old_name = item.get('name')
    new_name = item.get('new_name')
    alias = item.get('alias')


    arcpy.management.AlterField(in_table='ms_stations_clip', 
                                field=old_name, 
                                new_field_name = new_name,
                                new_field_alias = alias)

Alter the year field alias

In [ ]:
fields = arcpy.ListFields('ms_stations_clip')

for field in fields:
    if field.name.startswith('y'):
        year = field.name[2:]
        alias = '{} Avg Volume Count'.format(year)
        print(alias)
        arcpy.management.AlterField(in_table='ms_stations_clip', 
                                field=field.name, 
                                new_field_alias =alias)                


Add URL field.

In [ ]:
fc = 'ms_stations_clip'

arcpy.AddField_management(in_table=fc, field_name='ms2_tcds_url', field_type='TEXT', field_length='1000', field_alias='MS2 TCDS URL')


arcpy.management.CalculateField(
    in_table="ms_stations_clip",
    field="ms2_tcds_url",
    expression='"https://adot-all.public.ms2soft.com/tcds/tsearch.asp?loc=Adot&mod=tcds&local_id="+!local_id!',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)



Alter fields.

In [ ]:
fields_to_alter = [('latest_count', 'latest_estimate', "Latest Estimate"),
                   ('latest_count_date', 'latest_estimate_date', "Latest Estimate Date")]

for old_name, new_name, new_alias in fields_to_alter:
   arcpy.management.AlterField(in_table='ms_stations_clip', 
                                field=old_name, 
                                new_field_name = new_name,
                                new_field_alias = new_alias)

Change field type

In [ ]:
arcpy.management.CalculateField(
    in_table='ms_stations_clip', 
    field='temp', 
    expression='!latest_estimate!', 
    field_type='LONG')

In [33]:
arcpy.DeleteField_management('ms_stations_clip', 'latest_estimate')

<Result './data/sun_cloud_ms2_revision.gdb\\ms_stations_clip'>

In [34]:
fields_to_alter = [('temp', 'latest_estimate', 'Latest Estimate')]

for old_name, new_name, new_alias in fields_to_alter:
   arcpy.management.AlterField(in_table='ms_stations_clip', 
                                field=old_name, 
                                new_field_name = new_name,
                                new_field_alias = new_alias)

Covert the date field.

In [36]:
arcpy.management.CalculateField(
    in_table='ms_stations_clip', 
    field='temp', 
    expression='!latest_estimate_date!', 
    field_type='DATE')



<Result './data/sun_cloud_ms2_revision.gdb\\ms_stations_clip'>

In [38]:
arcpy.DeleteField_management('ms_stations_clip', 'latest_estimate_date')
fields_to_alter = [('temp', 'latest_estimate_date', 'Latest Estimate Date')]
for old_name, new_name, new_alias in fields_to_alter:
   arcpy.management.AlterField(in_table='ms_stations_clip', 
                                field=old_name, 
                                new_field_name = new_name,
                                new_field_alias = new_alias)